<a href="https://colab.research.google.com/github/nerdk312/Multimodal_VQ-VAE/blob/master/Hierarchical_layers_Updated_decoder_261219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, we'll mount Drive.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


I happen to have an existing `.py` file in Drive.

In [0]:
!ls /content/gdrive/My\ Drive/vq-vae-2-master_joint_modality/vq_vae_2/examples

In [0]:
!cat '/content/gdrive/My Drive/vq-vae-2-master/vq_vae_2/vq_vae.py'

We'll need to update our path to import from Drive.

In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/vq-vae-2-master_multi_modality')

Now we can import the library and use the function.

In [0]:

import itertools
import os

from PIL import Image
from keras.preprocessing import image as image_utils
import numpy as np
import torch
import torchvision
import torch.optim as optim

from vq_vae_2.examples.Two_layers_combined.model import make_vq_vae
target_size = (128,128)

Using TensorFlow backend.


In [0]:
inp_dir = '/content/gdrive/My Drive/thermal_8_bit'
image_list = os.listdir(inp_dir)
all_images = []

rgb_inp_dir = '/content/gdrive/My Drive/RGB'
rgb_image_list = os.listdir(rgb_inp_dir)
all_rgb_images = []

quick_test = False
desired_examples = 8300
if quick_test:
  for i, img_name in enumerate(image_list):
    if i< 10:
      fname = inp_dir + '/' + img_name
      image = image_utils.load_img(fname, color_mode='grayscale').resize(target_size,Image.ANTIALIAS)
      image = np.array(image.getdata()).reshape(target_size[0], target_size[1],1)
      image = image.astype('float32')/255
      all_images.append(image)
      
    
  for j, img_name in enumerate(rgb_image_list):
    if j < 10:
      fname = rgb_inp_dir + '/' + img_name
      image = image_utils.load_img(fname).resize(target_size,Image.ANTIALIAS)
      image = np.array(image.getdata()).reshape(target_size[0], target_size[1],3)
      image = image.astype('float32')/255
      all_rgb_images.append(image)
      
    else:
      pass
else:
  
  for i, img_name in enumerate(image_list):
    if i < desired_examples:
      fname = inp_dir + '/' + img_name
      image = image_utils.load_img(fname, color_mode='grayscale').resize(target_size,Image.ANTIALIAS)
      image = np.array(image.getdata()).reshape(target_size[0], target_size[1],1)
      image = image.astype('float32')/255
      all_images.append(image)
      if not i%100:
        print(i)
    else: 
      pass
  
  for j, img_name in enumerate(rgb_image_list):
    if j < desired_examples:
      fname = rgb_inp_dir + '/' + img_name
      image = image_utils.load_img(fname).resize(target_size,Image.ANTIALIAS)
      image = np.array(image.getdata()).reshape(target_size[0], target_size[1],3)
      image = image.astype('float32')/255
      all_rgb_images.append(image)
      if not j%100: # Nawid  -indicates how far through it is in the process
        print(j)
    else:
      pass
  
np.save('Thermal_'+str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy',all_images)
np.save('RGB_' +str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy',all_rgb_images)

# UPLOADING FROM DRIVE DIRECTLY

In [0]:
upload_directly = True
if upload_directly:
  x_rgb = np.load('/content/gdrive/My Drive/RGB_'+str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy')
  x = np.load('/content/gdrive/My Drive/Thermal_'+str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy')
else:
  x_rgb = np.load('/content/RGB_'+str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy')
  x = np.load('/content/Thermal_'+str(target_size[0]) + 'x' + str(target_size[1]) + '_images.npy')

print(x_rgb.shape)
print(x.shape)

(8350, 128, 128, 3)
(8350, 128, 128, 1)


Making predictions

In [0]:
BATCH_SIZE = 16
def load_images(input_x): # Nawid - Loads images
    while True:
        for data in create_data_loader(input_x):
          #data = data.float()/255 # Nawid - Divides by 255 
          data = data.permute(0, 3, 1, 2).contiguous() # Nawid - Changes into pytorch ordering
          yield data
       
def create_data_loader(input_x): # Nawid - Creates a dataloader object
    return torch.utils.data.DataLoader(input_x, batch_size=BATCH_SIZE, shuffle=True)


device = torch.device('cuda')
VAE_PATH = 'vae.pt'

def main():
  model = make_vq_vae()
  if os.path.exists(VAE_PATH):
    model.load_state_dict(torch.load(VAE_PATH, map_location='cpu'))
  model.to(device)
  optimizer = optim.Adam(model.parameters())
  i = 0
  for x_rgb_batch, x_thermal_batch in zip(load_images(x_rgb), load_images(x)):
    x_rgb_batch= x_rgb_batch.to(device)
    x_thermal_batch =  x_thermal_batch.to(device)
    terms = model(x_rgb_batch, x_thermal_batch)
    #print('step %d: mse=%f mse_top=%f mse_top_thermal=%f' %
    #      (i, terms['losses'][-1].item(), terms['losses'][0].item(),terms['losses_thermal'][0].item()))
    
    if not i % 50:
      print('step %d: mse=%f mse_top=%f mse_top_thermal=%f' %
            (i, terms['losses'][-1].item(), terms['losses'][0].item(),terms['losses_thermal'][0].item()))
    
    optimizer.zero_grad() #Nawid - I think this makes the gradient become zero
    terms['loss'].backward() # Nawid - Backpropagates the total loss term ( which is related to all the different components)
    optimizer.step() # Nawid- Updates using optimizer
    model.revive_dead_entries()
    i +=1
    if not i % 100:
      torch.save(model.state_dict(), VAE_PATH)
      save_reconstructions(model,x_rgb_batch, x_thermal_batch) # Nawid - saves the reconstruction




def save_reconstructions(vae,rgb_images, thermal_images):
  vae.eval()
  with torch.no_grad():
    #output, output_thermal = vae.full_reconstructions(rgb_images,thermal_images)
    #top_output, real_output = output
    #print("reconstruction output shape:",top_output.shape)
    recons,recons_thermal = vae.full_reconstructions(rgb_images,thermal_images)
    recons = [torch.clamp(x, 0, 1).permute(0, 2, 3, 1).detach().cpu().numpy()
                  for x in recons] # Nawid - Clamps all the different elements in recons 
    recons_thermal = [torch.clamp(x_thermal, 0, 1).permute(0, 2, 3, 1).detach().cpu().numpy()
                  for x_thermal in recons_thermal] # Nawid - Clamps all the different elements in recons_thermal
    #top_output, real_output = output
    #top_output_thermal, real_output_thermal = output_thermal
    #print('top_output shape:',top_output.shape)
    #print('real_output shape:',real_output.shape)
    #print('top_output_thermal shape:',top_output_thermal.shape)
    #print('real_output_thermal shape:',real_output_thermal.shape)

    #for x_rgb, x_thermal in vae.full_reconstructions(rgb_images,thermal_images):
      #print("Here is both input shapes",x_rgb.shape)
      #print("Here is x_thermal shape",x_thermal.shape)
      #print(x_thermal)
      #recons = [torch.clamp(x_rgb, 0, 1).permute(0, 2, 3, 1).detach().cpu().numpy()]
      #recons_thermal = [torch.clamp(x_thermal, 0, 1).permute(0, 2, 3, 1).detach().cpu().numpy()]
    #recons,recons_thermal = [torch.clamp(x, 0, 1).permute(0, 2, 3, 1).detach().cpu().numpy()
                  #for x_rgb,x_thermal in vae.full_reconstructions(rgb_images,thermal_images)] # Nawid - performs the full_reconstructions of the image from using the top latent map, as well as the top level latent map combined with the lower level latent map.
    vae.train() # Nawid - This sets the nn.module into training mode
    top_recons, real_recons = recons # Nawid - I believe the top_recons is the reconstruction using only the top latent map, whilst real_recons is using both the top latent map and the bottom latent map
    rgb_images = rgb_images.permute(0, 2, 3, 1).detach().cpu().numpy()
    columns = np.concatenate([top_recons, real_recons, rgb_images], axis=-2)
    columns = np.concatenate(columns, axis=0)
    Image.fromarray((columns * 255).astype('uint8')).save('reconstructions_rgb.png') # Nawid - Changes the valeus back to 255 and integers and save the reconstructions


    top_recons_thermal,real_recons_thermal = recons_thermal
    thermal_images = thermal_images.permute(0,2,3,1).detach().cpu().numpy()
    thermal_images = thermal_images.reshape(thermal_images.shape[0],thermal_images.shape[1],thermal_images.shape[2])

    top_recons_thermal = top_recons_thermal.reshape(top_recons_thermal.shape[0],top_recons_thermal.shape[1],top_recons_thermal.shape[2])
    real_recons_thermal = real_recons_thermal.reshape(real_recons_thermal.shape[0],real_recons_thermal.shape[1],real_recons_thermal.shape[2])

    columns_thermal = np.concatenate([top_recons_thermal, real_recons_thermal,thermal_images], axis=-1) # Nawid - I need to concatenate along -2 when using rgb and -1 when using grayscale and when using  When using grayscale images, to save the files using PIL, i need to make to remove the channel and so there is one less axis to examine
    columns_thermal = np.concatenate(columns_thermal, axis=0)
    Image.fromarray((columns_thermal * 255).astype('uint8')).save('reconstructions_thermal.png') # Nawid - Changes the valeus back to 255 and integers and save the reconstructions

    
main()

'''
def save_reconstructions(x_rgb_batch, x_thermal_batch,decoded_rgb, decoded_thermal):
  x_rgb_batch = x_rgb_batch.permute(0,2,3,1).detach().cpu().numpy()
  x_thermal_batch = x_thermal_batch.permute(0,2,3,1).detach().cpu().numpy()
  decoded_rgb = decoded_rgb.permute(0,2,3,1).detach().cpu().numpy()
  decoded_thermal = decoded_thermal.permute(0,2,3,1).detach().cpu().numpy()
  
  columns = np.concatenate([decoded_rgb, x_rgb_batch], axis=-2) # Nawid - I need to concatenate along -2 when using rgb and -1 when using grayscale and when using  When using grayscale images, to save the files using PIL, i need to make to remove the channel and so there is one less axis to examine
  columns = np.concatenate(columns, axis=0)
  Image.fromarray((columns * 255).astype('uint8')).save('reconstructions_rgb.png') # Nawid - Changes the valeus back to 255 and integers and save the reconstructions
  
  x_thermal_batch = x_thermal_batch.reshape(x_thermal_batch.shape[0],x_thermal_batch.shape[1],x_thermal_batch.shape[2]) # Nawid - For grayscale images, need to remove the channel in order to save data in a PIL forma
  decoded_thermal = decoded_thermal.reshape(decoded_thermal.shape[0],decoded_thermal.shape[1],decoded_thermal.shape[2])

  columns_thermal = np.concatenate([decoded_thermal, x_thermal_batch], axis=-1) # Nawid - I need to concatenate along -2 when using rgb and -1 when using grayscale and when using  When using grayscale images, to save the files using PIL, i need to make to remove the channel and so there is one less axis to examine
  columns_thermal = np.concatenate(columns_thermal, axis=0)
  Image.fromarray((columns_thermal * 255).astype('uint8')).save('reconstructions_thermal.png') # Nawid - Changes the valeus back to 255 and integers and save the reconstructions
'''

